In [ ]:
"""
1. Get data of books made into films based on 2 listopia "book to movie lists" on goodreads.

Scrape search result pages for the book link to individual book page.
Scrape book pages

Ouput to pickle 'book_data'

2. Use selenium to search for 'movie_title' on goodreads.

functions used: 
get_soup(url) from scrape.py
get_book_data(book) from get_book.py
search_goodreads(movie_title) from get_book.py

Output to pickle 'found_book'.


"""

In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

import pickle

import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from scrape import *
from get_book import *

## 1. Scrape from Listopia lists using BeautifulSoup

In [ ]:
# Create url list of the search results (19 pages)
# first list: (I Saw the Movie & Read the Book)

listopia_base_url="https://www.goodreads.com/list/show/2451.I_Saw_the_Movie_Read_the_Book"

listopia_url_list=[listopia_base_url]
for i in range(2,20):
    url = listopia_base_url + '?page=' + str(i)
    listopia_url_list.append(url)

In [ ]:
# Create soup_list (19 soup for 19 pages)
listopia_soup_list=[]
for url in listopia_url_list:
    listopia_soup_list.append(get_soup(url))

In [ ]:
# Find url for each book from the list (I Saw the Movie & Read the Book) webpages
# Try first page (listopia_base_url)

base_book_url="https://www.goodreads.com/"
book_url_list=[]

soup = listopia_soup_list[1]

table= soup.find('table', class_="tableList js-dataTooltip")
rows = [row for row in table.find_all('tr')]

for i in range(100):
    book_url = rows[i].find_all('a')[0].get('href')
    book_url_list.append(base_book_url+book_url)

In [ ]:
# Find url for each book from the list webpages
# Create a list of url for 1800+ books

book_url_list=[]
base_book_url="https://www.goodreads.com/"

for soup in listopia_soup_list[:-1]:
    table= soup.find('table', class_="tableList js-dataTooltip")
    rows = [row for row in table.find_all('tr')]

    for i in range(100):
        book_url = rows[i].find_all('a')[0].get('href')
        book_url_list.append(base_book_url+book_url)

In [ ]:
len(book_url_list)

In [ ]:
# Create soup_list for the books

book_soup_list=[]
for book_url in book_url_list:
    book_soup = get_soup(book_url)
    book_soup_list.append(book_soup)

In [ ]:
len(book_soup_list)

In [ ]:
# Test with Harry Potter #1
hp_url = book_url_list[0]
hp = book_soup_list[0]
get_book_data(hp)

In [ ]:
# Create list of book_dict
book_data=[]
for book_soup in book_soup_list:
    book = get_book_data(book_soup)
    book_data.append(book)

In [ ]:
len(book_data)

In [ ]:
# Create pd DataFrame of 1800 books!
book_df = pd.DataFrame(book_data)
book_df

In [ ]:
# Save the data to pickle file
book_df.to_pickle('../data/book_data')

## 2. Search movie title on goodreads using selenium

In [ ]:
# Load in movie_data picke file
movie_df = pd.read_pickle('../data/movie_data')
book_df = pd.read_pickle('../data/book_data')

In [ ]:
movie_list = list(movie_df.movie_title)
book_list = list(book_df.book_title)

In [ ]:
# need_book: movies that do not have a matching book
need_book_pre = list(set(movie_list).difference(set(book_list)))
need_book = [book.replace('&','and').replace(":",'') for book in need_book_pre]

In [ ]:
# append search result (book_dict) to the list found_book

found_book=[]
driver = webdriver.Chrome(chromedriver)
for movie in need_book:
    result = search_goodreads(movie)
    found_book.append(result)

In [ ]:
# convert the list to pd.DataFrame and pickle
found_book_df = pd.DataFrame(found_book)
found_book_df.to_pickle('../data/found_book')